# 03장. 크롤링 시작하기

- 크롤링의 핵심은 **재귀**

In [19]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

In [2]:
html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")

In [4]:
bs0bj = BeautifulSoup(html, "html.parser")

In [5]:
for link in bs0bj.findAll("a"):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#p-search
/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
http://baconbros.com/
#cite_note-1
#cite_note-actor-2
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Awards
/wiki/Primetime_Emmy_Award
/wiki/The_Guardian
/wiki/Academy_Award
#cite_note-3
/wiki/Hollywood_Walk_of_Fame
/wiki/Social_networks
/wiki/Six_Degrees_of_Kevin_Bacon
/wiki/SixDegrees.org
#cite_note-walk-4
#Early_life_and_education
#Acting_career
#Early_work
#1980s
#1990s
#2000s
#2010s
#Advertising_work
#Personal_life
#Six_Degrees_of_Kevin_Bacon
#Music
#Awards_and_nominations
#Refere

- id가 bodyContent인 div 안에 있음
- url에는 세미콜론이 포함되어 있지 않음
- /wiki/로 시작

In [18]:
for link in bs0bj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Kevin_Bacon_(disambiguation)
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Awards
/wiki/Primetime_Emmy_Award
/wiki/The_Guardian
/wiki/Academy_Award
/wiki/Hollywood_Walk_of_Fame
/wiki/Social_networks
/wiki/Six_Degrees_of_Kevin_Bacon
/wiki/SixDegrees.org
/wiki/Philadelphia
/wiki/Edmund_Bacon_(architect)
/wiki/Pennsylvania_Governor%27s_School_for_the_Arts
/wiki/Bucknell_University
/wiki/Glory_Van_Scott
/wiki/Kevin_Bacon_filmography
/wiki/Circle_in_the_Square
/wiki/Nancy_Mills
/wiki/Cosmopolitan_(magazine)
/wiki/Fraternities_and_sororities
/wiki/Animal_House
/wiki/Search_for_Tomorrow
/wiki/Guiding_Light
/wiki/

In [21]:
random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    html = urlopen("http://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$"))

In [22]:
%%time
links = getLinks("/wiki/Kevin_Bacon")
while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)

/wiki/Ra%C3%BAl_Juli%C3%A1
/wiki/Paul_Giamatti
/wiki/Thomas_Haden_Church
/wiki/Maggie_Smith
/wiki/Susan_Wooldridge
/wiki/Liz_Smith_(actress)
/wiki/Desert_Island_Discs
/wiki/Ladies_of_Letters
/wiki/The_Archers
/wiki/Ned_Sherrin
/wiki/Order_of_the_British_Empire
/wiki/Air_Force_Cross_(United_Kingdom)
/wiki/Indian_Order_of_Merit
/wiki/United_Kingdom
/wiki/Hungary
/wiki/Kingdom_of_Hungary_in_the_Middle_Ages
/wiki/Charles_VI,_Holy_Roman_Emperor
/wiki/Wenceslaus_I,_Duke_of_Bohemia
/wiki/Nativity_Fast
/wiki/Vespers
/wiki/Reform_of_the_Roman_Breviary_by_Pope_Pius_X
/wiki/Epiphany_(Christian)
/wiki/Moscow_Kremlin
/wiki/Catherine_Palace_(Moscow)
/wiki/Catherine_Palace
/wiki/Kolomenskoye
/wiki/Tented_roof
/wiki/Kolomenskoye
/wiki/Central_Federal_District
/wiki/Autonomous_okrugs_of_Russia
/wiki/World_War_II
/wiki/Phoney_war
/wiki/Blitzkrieg
/wiki/Battle_of_Narva_(1944)
/wiki/Western_Desert_Campaign
/wiki/Military_history_of_Finland_during_World_War_II
/wiki/Cinema_of_Finland
/wiki/Cinema_of_Lebano

KeyboardInterrupt: 

In [23]:
pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    try:
        print(bsObj.h1.get_text())
        print(bsObj.find(id ="mw-content-text").findAll("p")[0])
        print(bsObj.find(id="ca-edit").find("span").find("a").attrs['href'])
    except AttributeError:
        print("This page is missing something! No worries though!")
    
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                #We have encountered a new page
                newPage = link.attrs['href']
                print("----------------\n"+newPage)
                pages.add(newPage)
                getLinks(newPage)

In [24]:
%%time
getLinks("") 

Main Page
<p>The <b><a href="/wiki/Impala" title="Impala">impala</a></b> (<i>Aepyceros melampus</i>) is a medium-sized <a href="/wiki/Antelope" title="Antelope">antelope</a> in eastern and southern Africa. First <a class="mw-redirect" href="/wiki/Scientific_description" title="Scientific description">described</a> by German zoologist <a href="/wiki/Hinrich_Lichtenstein" title="Hinrich Lichtenstein">Hinrich Lichtenstein</a> in 1812, it reaches 70–92 centimetres (28–36 inches) at the shoulder, weighs 40–76 kilograms (88–168 pounds), and has a glossy, reddish brown coat. The male's slender, <a href="/wiki/Lyre" title="Lyre">lyre</a>-shaped <a href="/wiki/Horn_(anatomy)" title="Horn (anatomy)">horns</a> can reach 45–92 centimetres (18–36 in), measured along the curve. Active mainly during the day, the impala may be gregarious or <a href="/wiki/Territory_(animal)" title="Territory (animal)">territorial</a>. Three distinct social groups can be observed – the territorial males, bachelor herds

KeyboardInterrupt: 

# Scrapy 사용 => 공식 문서를 읽어볼 것! 현재 윈도우에선 python 2.7만 지원